<a href="https://colab.research.google.com/github/AlexanderVinarsky/IT_CUP/blob/main/ITcup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [596]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [597]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [598]:
import json
import gensim.downloader as api
import numpy as np
import torch
import string
import transformers
import gensim
from transformers import DistilBertModel, DistilBertTokenizer, logging

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt

nltk.download('stopwords')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [599]:
with open("drive/MyDrive/ITcup/ranking_train.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

In [600]:
data=data[:1000]

In [601]:
stop_words = stopwords.words('english')
for i in range(len(stop_words)):
  stop_words[i] = "".join([char for char in stop_words[i] if char not in "'"])

porter = PorterStemmer()

In [602]:
nltk.download('punkt')
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [603]:
scores   = []
for i in range(len(data)):
  for j in range(len(data[i]['comments'])):
    scores.append(data[i]['comments'][j]['score'])

exclamations = []
questions = []
links = []
lengths = []
for i in range(len(data)):
  avg_com_len=0
  for j in range(len(data[i]['comments'])):
    avg_com_len += len(data[i]['comments'][j]['text'])
  avg_com_len/=5
  tmp_link = 0
  tmp_length = 0
  for j in range(len(data[i]['comments'])):
    tmp_length = len(data[i]['comments'][j]['text'])/avg_com_len
    tmp_exclamation = data[i]['comments'][j]['text'].count("!")#/tmp_length
    tmp_question = data[i]['comments'][j]['text'].count("?")#/tmp_length
    if ("http" or "www") in data[i]['comments'][j]['text']:
      tmp_link += 1

    exclamations.append(tmp_exclamation)
    questions.append(tmp_question)
    links.append(tmp_link)
    lengths.append(tmp_length)

In [604]:
import pandas as pd
df = pd.DataFrame(list(zip(scores[:len(data)*5], exclamations, questions, links, lengths)), columns =['Score', '!', '?', 'Has Links', 'Length / Avg len in post'])

In [605]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Score,!,?,Has Links,Length / Avg len in post
Score,1.000000,-0.046498,-0.075514,0.281478,-0.433200
!,-0.046498,1.000000,0.139172,-0.020778,0.118079
?,-0.075514,0.139172,1.000000,-0.019201,0.208652
Has Links,0.281478,-0.020778,-0.019201,1.000000,-0.117209
Length / Avg len in post,-0.433200,0.118079,0.208652,-0.117209,1.000000


In [606]:
for datum in data:
  datum['text'] = datum['text'].lower()
  datum['text'] = "".join([char for char in datum['text'] if char not in string.punctuation])
  datum['text'] = word_tokenize(datum['text'])
  datum['text'] = [word for word in datum['text'] if word not in stop_words]
  #datum['text'] = [porter.stem(word) for word in datum['text']]
  #datum['text'] = ' '.join(datum['text'])
  for comment in datum['comments']:
    comment['text'] = comment['text'].lower()
    comment['text'] = "".join([char for char in comment['text'] if char not in string.punctuation])
    comment['text'] = word_tokenize(comment['text'])
    comment['text'] = [word for word in comment['text'] if word not in stop_words]
    #comment['text'] = [porter.stem(word) for word in comment['text']]
    #comment['text'] = ' '.join(comment['text'])

In [607]:
texts = []
comments = []

for i in range(len(data)):
  texts.append(data[i]['text'])
  for j in range(len(data[i]['comments'])):
    comments.append(data[i]['comments'][j]['text'])

In [608]:
for i in range(len(comments)):
  if len(comments[i]) == 0:
    print(i)
    comments[i].append('house');
for i in range(len(texts)):
  if len(texts[i]) == 0:
    print(i)
    texts[i].append('house');

84
429
714
3124


In [609]:
from gensim.models import Word2Vec
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

In [610]:
similarity=[]

sentences=comments[0:5]
target_sentence=texts[0]

for i in range(len(texts)):
  sentences_similarity = np.zeros(len(sentences))
  w2v_vocab = model.vocab.keys()
  sentences=comments[5*i:5+5*i]
  target_sentence=texts[i]
  for idx, sentence in enumerate(sentences):
    sentence_words = [w for w in sentence if w in w2v_vocab]
    if (len(sentence_words) == 0):
      sentence_words=['fortune']
    target_sentence_words = [w for w in target_sentence if w in w2v_vocab]
    if (len(target_sentence_words) == 0):
      target_sentence_words=['fortune']
    sim = model.n_similarity(target_sentence_words, sentence_words)
    sentences_similarity[idx] = sim

  result = list(zip(sentences_similarity))
  for i in range(5):
    similarity+=result[i]

**Корреляция между скором и длиной, а также между скором и наличием ссылок в комментарии более значимые, чем прочие**

In [611]:
df['Similarity'] = similarity
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Score,!,?,Has Links,Length / Avg len in post,Similarity
Score,1.000000,-0.046498,-0.075514,0.281478,-0.433200,-0.089887
!,-0.046498,1.000000,0.139172,-0.020778,0.118079,-0.004847
?,-0.075514,0.139172,1.000000,-0.019201,0.208652,0.029056
Has Links,0.281478,-0.020778,-0.019201,1.000000,-0.117209,-0.062523
Length / Avg len in post,-0.433200,0.118079,0.208652,-0.117209,1.000000,0.166566
Similarity,-0.089887,-0.004847,0.029056,-0.062523,0.166566,1.000000


In [612]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [613]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(200, 64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(64,32),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(32,16),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(16,5)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [614]:
model = api.load("glove-wiki-gigaword-100")
w2v_vocab = model.vocab.keys()

In [615]:
comms=[]
for i in range(len(comments)):
  if (comments[i] and comments[i][0] in w2v_vocab):
    cur = model.wv.get_vector(comments[i][0])
  else:
    cur = model.wv.get_vector('the')
    for j in range(1, len(comments[i])):
      if (comments[i][j] and comments[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
      else:
        cur = model.wv.get_vector('the')
  comms.append(cur/(len(comments[i])))

<ipython-input-615-c53b47995baa>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(comments[i][0])
<ipython-input-615-c53b47995baa>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector('the')
<ipython-input-615-c53b47995baa>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
<ipython-input-615-c53b47995baa>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector('the')


In [616]:
posts=[]
for i in range(len(texts)):
  if (texts[i] and texts[i][0] in w2v_vocab):
    cur = model.wv.get_vector(texts[i][0])
  else:
    cur = model.wv.get_vector('the')
    for j in range(1, len(texts[i])):
      if (texts[i][j] and texts[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)
      else:
        cur = model.wv.get_vector('the')
  posts.append(cur/(len(texts[i])))
posts = [x for x in posts for i in range(5)]

<ipython-input-616-ed2a14830e8b>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(texts[i][0])
<ipython-input-616-ed2a14830e8b>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector('the')
<ipython-input-616-ed2a14830e8b>:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)
<ipython-input-616-ed2a14830e8b>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector('the')


In [617]:
expected = [
([1,0,0,0,0],
[0,1,0,0,0],
[0,0,1,0,0],
[0,0,0,1,0],
[0,0,0,0,1],)*(int(len(comms)/5)+5)
]
expected[0] = expected[0][:len(comms)]

In [618]:
import torch
import torch.nn as nn
import torch.optim as optim

In [619]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [620]:
!pip install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [621]:
loss_fn = nn.CrossEntropyLoss()

model = NeuralNetwork()
optimizer = optim.SGD(model.parameters(), lr=0.05)

for epoch in range(3):
  for i in tqdm(range(len(expected[0]))):
    outputs = model.forward(torch.FloatTensor(np.concatenate((comms[i], posts[i]))).requires_grad_().flatten())
    expected_flat = torch.FloatTensor(expected[0][i]).requires_grad_().flatten()
    loss = loss_fn(outputs, expected_flat)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pass

100%|██████████| 5000/5000 [00:07<00:00, 638.60it/s]


In [622]:
class_prob = []
for i in range(len(comms)):
  outputs = model.forward(torch.FloatTensor(np.concatenate((comms[i], posts[i]))).flatten())
  class_prob.append(outputs.tolist())

In [623]:
scores_final_train = []
for k in range(int(len(class_prob)/5)):
  class_prob_batch = class_prob[k:k+5].copy()
  answers=[]
  for i in range(0,5):
    max=-2
    index=None
    for cur_index in range(5):
      if (cur_index not in answers):
        if(class_prob_batch[i][cur_index]>max):
          max = class_prob_batch[i][cur_index]
          index=cur_index
    answers.append(index)
  scores_final_train+=answers
print(scores_final_train)



[0, 1, 2, 3, 4, 1, 0, 3, 4, 2, 1, 0, 4, 2, 3, 1, 4, 0, 3, 2, 4, 0, 1, 3, 2, 0, 1, 3, 4, 2, 1, 0, 4, 3, 2, 1, 4, 0, 3, 2, 4, 0, 1, 2, 3, 0, 1, 2, 3, 4, 1, 2, 0, 3, 4, 2, 1, 0, 3, 4, 1, 0, 2, 3, 4, 1, 0, 2, 3, 4, 1, 0, 2, 3, 4, 1, 0, 3, 2, 4, 0, 1, 2, 4, 3, 1, 0, 4, 2, 3, 0, 4, 2, 1, 3, 4, 2, 1, 0, 3, 4, 1, 0, 3, 2, 1, 0, 4, 2, 3, 1, 4, 2, 0, 3, 4, 2, 0, 1, 3, 2, 0, 1, 3, 4, 0, 1, 3, 2, 4, 1, 0, 3, 2, 4, 1, 0, 2, 4, 3, 1, 0, 4, 3, 2, 0, 4, 3, 2, 1, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 1, 0, 2, 4, 1, 0, 3, 2, 1, 0, 3, 2, 4, 1, 0, 3, 2, 4, 1, 0, 3, 2, 4, 1, 0, 2, 3, 4, 1, 0, 2, 3, 4, 0, 1, 3, 2, 4, 0, 1, 2, 4, 3, 1, 0, 4, 3, 2, 0, 4, 1, 3, 2, 4, 1, 3, 0, 2, 1, 3, 0, 2, 4, 3, 0, 1, 2, 4, 0, 1, 2, 3, 4, 1, 0, 3, 2, 4, 1, 0, 3, 2, 4, 1, 0, 2, 3, 4, 1, 0, 2, 4, 3, 0, 1, 4, 3, 2, 0, 4, 1, 2, 3, 4, 1, 0, 2, 3, 1, 0, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 3, 2, 4, 0, 1, 2, 3, 4, 1, 0, 2, 3, 4, 0, 1, 3, 2, 4, 0, 1, 3, 2, 4, 1, 0, 2, 3, 4, 1, 0, 2, 3, 4, 0, 1, 3, 2, 4, 0, 1, 3, 

In [624]:
counter=0
for i in range(len(scores)):
  if (scores[i] == scores_final_train[i]):
    counter+=1
res = counter/len(scores)
res

0.383

# **Займёмся Тестовым Датасетом:**

In [625]:
with open("drive/MyDrive/ITcup/ranking_test.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f]

In [626]:
stop_words = stopwords.words('english')
for i in range(len(stop_words)):
  stop_words[i] = "".join([char for char in stop_words[i] if char not in "'"])

porter = PorterStemmer()

In [627]:
for datum in test_data:
  datum['text'] = datum['text'].lower()
  datum['text'] = "".join([char for char in datum['text'] if char not in string.punctuation])
  datum['text'] = word_tokenize(datum['text'])
  datum['text'] = [word for word in datum['text'] if word not in stop_words]
  for comment in datum['comments']:
    comment['text'] = comment['text'].lower()
    comment['text'] = "".join([char for char in comment['text'] if char not in string.punctuation])
    comment['text'] = word_tokenize(comment['text'])
    comment['text'] = [word for word in comment['text'] if word not in stop_words]

In [628]:
test_texts = []
test_comments = []

for i in range(len(test_data)):
  test_texts.append(test_data[i]['text'])
  for j in range(len(test_data[i]['comments'])):
    test_comments.append(test_data[i]['comments'][j]['text'])

In [629]:
for i in range(len(test_comments)):
  if len(test_comments[i]) == 0:
    print(i)
    test_comments[i].append('house');
for i in range(len(test_texts)):
  if len(test_texts[i]) == 0:
    print(i)
    test_texts[i].append('house');

1935
1939
22429
42702
5422


In [630]:
nltk.download('punkt')
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [631]:
test_scores = []
for i in range(len(test_data)):
  for j in range(len(test_data[i]['comments'])):
    test_scores.append(test_data[i]['comments'][j]['score'])

In [632]:
model1 = api.load("glove-wiki-gigaword-100")
w2v_vocab = model1.vocab.keys()

In [635]:
test_comms=[]
for i in range(len(test_comments)):
  if (test_comments[i] and test_comments[i][0] in w2v_vocab):
    cur = model1.wv.get_vector(test_comments[i][0])
    for j in range(1, len(test_comments[i])):
      if (test_comments[i][j] and test_comments[i][j] in w2v_vocab):
        cur = np.sum([cur, model1.wv.get_vector(test_comments[i][j])], axis = 0)
  test_comms.append(cur/(len(test_comments[i])))

<ipython-input-635-665e1c763004>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model1.wv.get_vector(test_comments[i][0])
<ipython-input-635-665e1c763004>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model1.wv.get_vector(test_comments[i][j])], axis = 0)


In [645]:
test_posts=[]
for i in range(len(test_texts)):
  if (test_texts[i] and test_texts[i][0] in w2v_vocab):
    cur = model1.wv.get_vector(test_texts[i][0])
    for j in range(1, len(test_texts[i])):
      if (test_texts[i][j] and test_texts[i][j] in w2v_vocab):
        cur = np.sum([cur, model1.wv.get_vector(test_texts[i][j])], axis = 0)
  test_posts.append(cur/(len(test_texts[i])))
test_posts = [x for x in test_posts for i in range(5)]

<ipython-input-645-4d5a25c4d9d4>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model1.wv.get_vector(test_texts[i][0])
<ipython-input-645-4d5a25c4d9d4>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model1.wv.get_vector(test_texts[i][j])], axis = 0)


In [646]:
test_class_prob = []
for i in range(len(test_comms)):
  test_outputs = model.forward(torch.FloatTensor(np.concatenate((test_comms[i], test_posts[i]))).flatten())
  test_class_prob.append(test_outputs.tolist())

In [648]:
test_scores_final_train = []
for k in range(int(len(test_class_prob)/5)):
  test_class_prob_batch = test_class_prob[k:k+5].copy()
  test_answers=[]
  for i in range(0,5):
    max=-2
    index=0
    for cur_index in range(5):
      if (cur_index not in test_answers):
        if(test_class_prob_batch[i][cur_index]>max):
          max = test_class_prob_batch[i][cur_index]
          index=cur_index
    test_answers.append(index)
  test_scores_final_train+=test_answers
print(test_scores_final_train)

[0, 4, 3, 2, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 1, 2, 0, 4, 1, 3, 2, 0, 1, 4, 3, 2, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4, 2, 3, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 0, 1, 4, 3, 2, 1, 0, 4, 2, 3, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 

In [659]:
with open("drive/MyDrive/ITcup/ranking_test.jsonl", "r") as f:
    result = [json.loads(line) for line in f]

In [660]:
for i in range(len(result)):
  for j in range(len(result[i]['comments'])):
    result[i]['comments'][j]['score'] = test_scores_final_train[i*5+j]